In [154]:
import pandas as pd
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
import numpy as np

In [128]:
url = "https://nlp100.github.io/data/neko.txt"
df = pd.read_csv(url, header = None)
df.rename(columns={0 : "raw"}, inplace=True)

indices = [df.index[df.raw == s][0] for s in ["一", "二", "三", "四", "五", "六", "七", "八", "九", "十", "十一"]]
indices.append(len(df))
chapter = []
for i in range(1, len(indices)):
    chapter.extend([i] * (indices[i] - indices[i-1]))

df["chapter"] = chapter
df.drop(index = indices[0:-1], inplace = True)
df.reset_index(drop=True, inplace = True)

In [129]:
tokenizer = Tokenizer()
token_filters = [POSStopFilter(["記号", "助詞", "助動詞"])]
a = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [130]:
def get_tokens(raw):
    test_tokens = a.analyze(raw)
    text = " ". join([t.base_form for t in test_tokens])
    return text
    

In [131]:
df["text"] = df["raw"].apply(get_tokens)

## Bag of Words
Cosine Similarity

In [171]:
counter = CountVectorizer()
vec_bow = counter.fit_transform(df["text"])
df["vec_bow"] = [vec for vec in vec_bow.toarray()]

tmp = df.groupby("chapter").vec_bow.apply(np.sum)
dict_bow = {i+1:x for i, x in enumerate(tmp)}
df["vec_bow_chap"] = df["chapter"].map(dict_bow)

df["similarity"] = df.apply(lambda x: spatial.distance.cosine(x.vec_bow, x.vec_bow_chap), axis = 1)

/opt/conda/lib/python3.9/site-packages/scipy/spatial/distance.py:630: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [180]:
df[df.groupby("chapter")["similarity"].rank(ascending = False) <= 5].groupby("chapter")["raw"].apply("".join)

chapter
1     そのうち池の上をさらさらと風が渡って日が暮れかかる。涎を本の上へ垂らす。第一色が違う。彼は純...
2              「何おめでてえ？球は上へ上へとのぼる。彼は強い。球が空にあがる。「御苦労だった。
3     「鼠どころじゃございません。シモニジスは八十で妙詩を作った。こいつは剣呑だと水桶の裏へかくれ...
4     ——もってのほかの事である。宜しゅうございます」「かしこまりました。「いやー珍客だね。「大変...
5      鼻は猫にとっても急所である。痛む事おびただしい。「七時半でしたろう」「黒足袋が一足」「頑愚だな。
6     鳳眠を驚かし奉ってはなはだ相済まん。いわゆる霊の交換だね。御気に入りませんかね。倦んじて薫ず...
7        図太い奴だ。台湾の生蕃である。和唐内の時にゃ無かったね。魔中の大王だ。「字って横文字ですか」
8                ずるい奴だ。大気※である。「にせものだろう。何かご用で」「君うるさくないか。
10    声盲だって片輪に違いない。夏だって夜露は毒だ。情ない事だ。「天探女でしょう。無類ですよ」「う...
Name: raw, dtype: object